In [ ]:
# from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/gdrive/')

# 병명증상정보.csv : "병명/증상 정보 data"를 csv로 변환한 파일
df = pd.read_csv("/content/gdrive/MyDrive/멀티캠퍼스_DS_DE_33/2nd_pjt/병명증상정보.csv", encoding='cp949')
df.head()

Mounted at /content/gdrive/


,담당자,부위,증상,질환명,진료과,TAG
0,박장훈,복부,신생아의 98%는 출생 후 24시간 내에 짙은 녹색의 태변을 배설합니다. 아기의 항...,선천성 거대결장증,소아외과,"히르쉬스프룽병 ,선청성대장질환 ,거대결장 ,대장조영술검사"
1,박장훈,복부,간에서 만들어진 담즙은 담도를 통해 소장으로 분비돼 소화를 돕는 데 사용됩니다. 그...,담도폐쇄증,"소화기내과, 간담췌외과, 소아외과","담도, 담도폐쇄증 ,billiary atresia ,간경변증 ,간이식 ,담도조영술 ..."
2,박장훈,복부,"조기 간암의 증상으로 피로감, 전신 쇠약감, 소화 불량, 상복부에 느껴지는 뻐근함,...",간암,"간담췌외과, 방사선종양학과, 혈액종양내과, 소화기내과, 간암센터","피로감, 구토 ,복통 ,황달 ,메스꺼움 ,식욕부진 ,체중감소 ,AFP ..."
3,박장훈,복부,"대표적인 증상으로는 복통, 설사, 체중 감소를 들 수 있습니다. 특히 10~20대 ...",크론병 Crohn disease,"소화기내과, 소아소화기영양과","복통 ,설사 ,식욕부진 ,체중감소 ,치루 ,장폐색 국소성 장염 ,궤양성 ..."
4,박장훈,복부,비만은 체내 근육량에 비해 지방조직이 과다한 상태를 의미합니다. 대한비만학회에서는 ...,고도비만 Severe obesity,"내분비내과, 가정의학과, 내분비당뇨병센터, 비만대사수술센터","빈혈, 속 쓰림 ,오심, 구토 ,복통 ,체중감소 ,위암개복수술 ,위암합병증 ..."


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
pip install soynlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
# soynlp을 이용하여 학습 이후, 응집도를 계산 해보자

import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

# 훈련 데이터를 다수의 문서로 분리
corpus = DoublespaceLineCorpus("2016-10-20.txt")

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 0.867 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [ ]:
# n-gram 어휘 벡터 생성
def getNgramWord(N,txt):
    txt = txt.split()
    ngrams = [txt[i:i+N] for i in range(len(txt)-N+1)]
    return ngrams

In [ ]:
# 응집도추출

for i in df['증상'].head(3):

  # 단어를 몇 개까지 이어 붙일지 결정
  for j in range(1,10):

    # 현재 평가하려는 단어들의 리스트, 그중에서 응집도가 가장 높은 항목만 출력
    que = []
    for ngramwords in getNgramWord(j,i):
      for word in ngramwords:
        print(word_score_table[word].cohesion_forward)
    else:
      que.sort(key=lambda x : x[1])

# 이 방법으로 평가를 하려면, 평가 하고 싶은 단어가 train data 내에 있어야 평가가 가능하다...

KeyError: '신생아의'

In [ ]:
print(word_score_table['번동파출소에서'].cohesion_forward)
# 이방법의 문제점은 여러 어절이 아닌, 한 어절 형태로만 평가가 가능하다...

0.4894032716126072


### 우선 N-gram으로 도출한 어절의 가장 마지막 단어의 형태소 분리를 해보자...
- 추후 찾아봐야 할 사항, TF-IDF, 코사인 유사도 etc.

In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 1.7 MB/s eta 0:00:00


### 형태소 분리 모델

In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab
import time

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()
# mecab = Mecab()

# 형태소 분석에 사용될 문장 예시
sentence = "히어로들이 세상에 존재하지 않는다는 사실을 깨달은 것은 상당히 시간이 지난뒤의 일이었다."


In [ ]:
# N-gram 된 문장에서, 가장 마지막 어구의 형태소를 분리하여 명사형태로 만들기
# ex1. 히어로들이 세상에 >> 히어로들이 세상
# ex2. 아기들의 복부가 팽창합니다 >> 아기들의 복부가 팽창
# 예시 형태로 변경하는 알고리즘을 구성해보자.

# hannanum

tmp1 = []
a = time.time()

for i in df['증상'].head(3):

  # 단어를 몇 개까지 이어 붙일지 결정
  for j in range(1,10):

    for k in range(len(getNgramWord(j,i))):
      # 정상 출력되는지 확인용
      # print(getNgramWord(j,i)[k], hannanum.nouns(str(getNgramWord(j,i)[k][-1])))

      change_word = hannanum.nouns(str(getNgramWord(j,i)[k][-1]))


      tmp2 = getNgramWord(j,i)[k][:j - 1] + change_word
      print(getNgramWord(j,i)[k])
      print(tmp2)
      print()
b = time.time()

print("총 소요 시간 : ",b-a,"초")
# 총 소요 시간 :  3.1711976528167725 초

# hannanum은 정교함이 살짝 떨어지는 것 같다.

In [ ]:
# kkma

tmp1 = []
a = time.time()
for i in df['증상'].head(3):

  # 단어를 몇 개까지 이어 붙일지 결정
  for j in range(1,10):

    for k in range(len(getNgramWord(j,i))):
      # 정상 출력되는지 확인용
      # print(getNgramWord(j,i)[k], hannanum.nouns(str(getNgramWord(j,i)[k][-1])))

      change_word = kkma.nouns(str(getNgramWord(j,i)[k][-1]))


      tmp2 = getNgramWord(j,i)[k][:j - 1] + change_word
      print(getNgramWord(j,i)[k])
      print(tmp2)
      print()
b = time.time()
print("총 소요 시간 : ",b-a,"초")
# 총 소요 시간 :  17.37988829612732 초

# kkma는 결과물이 마음에 들게 나오지 않는것 같다. 신뢰도를 높이기 위해 다양한 형태의 데이터를 제공하는 듯

In [ ]:
# komoran

tmp1 = []
a = time.time()
for i in df['증상'].head(3):

  # 단어를 몇 개까지 이어 붙일지 결정
  for j in range(1,10):

    for k in range(len(getNgramWord(j,i))):
      # 정상 출력되는지 확인용
      # print(getNgramWord(j,i)[k], hannanum.nouns(str(getNgramWord(j,i)[k][-1])))

      change_word = komoran.nouns(str(getNgramWord(j,i)[k][-1]))


      tmp2 = getNgramWord(j,i)[k][:j - 1] + change_word
      print(getNgramWord(j,i)[k])
      print(tmp2)
      print()
b = time.time()

print("총 소요 시간 : ",b-a,"초")
# 총 소요 시간 :  2.2232820987701416 초

# komoran도 결과물이 마음에 들게 나오지 않는것 같다. 지워지는 데이터가 너무 많다...

In [ ]:
# okt

tmp1 = []

a = time.time()

for i in df['증상'].head(1):

  # 단어를 몇 개까지 이어 붙일지 결정
  for j in range(1,2):

    for k in range(len(getNgramWord(j,i))):
      # 정상 출력되는지 확인용
      # print(getNgramWord(j,i)[k], hannanum.nouns(str(getNgramWord(j,i)[k][-1])))

      change_word = okt.nouns(str(getNgramWord(j,i)[k][-1]))


      tmp2 = getNgramWord(j,i)[k][:j - 1] + change_word
      print(getNgramWord(j,i)[k])
      print(tmp2)
      print()

b = time.time()

print("총 소요 시간 : ",b-a,"초")
# 총 소요 시간 :  1.711902379989624 초

# 가장 마지막에 거대결장증 처럼 나눠서 봄직한 단어가 나온다면 나눠 버리는 점만 마음에 들지 않는다.
# 전반적인 결과는 이게 가장 잘 나오는 것 같다.

# 속도도 가장 빠르다!!

In [ ]:
for i in df['증상'].head(3):
    print(okt.nouns(i))


['신생아', '출생', '후', '내', '녹색', '태변', '배설', '아기', '항문', '정상', '발달', '태변', '배설', '것', '선천성', '거대', '결장', '증', '의심', '수', '첫', '단서', '신생아', '시기', '이후', '평소', '배', '자주', '관장', '대변', '만성', '변비', '지속', '경우', '이', '병', '의심', '수', '신생아', '아기', '담즙', '구토', '팽창', '복부', '영양실조', '변비', '장', '등', '증상', '수', '소아', '청소년기', '치료', '만성', '변비', '진단', '기도']
['간', '담즙', '통해', '소장', '분비', '소화', '데', '사용', '담즙', '분비', '못', '간', '정체', '간', '손상', '간경변증', '진행', '담도폐쇄증', '가장', '특징', '증상', '달', '안', '아기', '황달', '회색', '빛', '대변', '신생아', '황달', '아주', '증상', '모유', '수유', '해도', '때문', '자칫', '위험', '신호', '치기', '따라서', '기저귀', '아기', '변', '색깔', '확인', '것', '아기', '황달', '대변', '색깔', '황금색', '회색', '흰색', '점차', '즉시', '소아과', '소아', '외과', '전문의', '담도폐쇄', '여부', '확인']
['조기', '간암', '증상', '피로', '감', '전신', '쇠약', '감', '소화', '불량', '상', '복부', '체중', '감소', '등', '꼽', '수', '증상', '간암', '암', '좀', '더', '진행', '간', '기능', '복수', '체중', '감소', '황달']
